In [2]:

import pandas as pd

train_df = pd.read_csv("/content/drive/MyDrive/SubTask-B-train.csv", encoding='latin-1')
test_df = pd.read_csv("/content/drive/MyDrive/SubTask-B-test.csv", encoding='latin-1')

train_df.head()  #used for previewing the dataset

print("Raw Dataset Shape:", train_df.shape)
train_df[['tweet', 'label']].head()


Raw Dataset Shape: (19019, 2)


,tweet,label
0,à¤ à¤®à¤° à¤à¤à¤¾à¤²à¤¾ à¤¦à¥à¤¨à¤¿à¤ à¤¸à...,0
1,à¤¬à¥à¤à¥à¤ªà¥ @BJP4India à¤à¥à¤¨à¥à¤¦à...,0
2,#AssemblyElections2022 à¤ªà¥à¤°à¤­à¤¾à¤°à¥ à...,0
3,#à¤¬à¥à¤°à¥à¤à¤¿à¤à¤ - à¤®à¤§à¥à¤¯à¤ªà¥...,0
4,#AmitShah à¤¨à¥ à¤à¤¹à¤¾ à¤à¤¤à¥à¤¤à¤° à¤ª...,0


In [1]:
!pip install -q torch transformers sentencepiece pandas scikit-learn emoji


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 12.5 MB/s eta 0:00:00


In [3]:
import pandas as pd
import re
import emoji
import numpy as np
import torch

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score

from transformers import (
    BertTokenizer,
    BertForSequenceClassification,
    Trainer,
    TrainingArguments
)


**Preprocess Data**

In [6]:
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r"http\S+|www\S+", "", text)
    text = re.sub(r"@\w+|#\w+", "", text)
    text = emoji.replace_emoji(text, replace="")
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    return text.strip()

train_df['clean_text'] = train_df['tweet'].apply(clean_text)
train_df = train_df[train_df['clean_text'] != ""]

test_df['clean_text'] = test_df['text'].apply(clean_text)
test_df = test_df[test_df['clean_text'] != ""]

**3: Train–Validation Split**

In [8]:
X_train, X_val, y_train, y_val = train_test_split(
    train_df['clean_text'],
    train_df['label'],
    test_size=0.2,
    random_state=42,
    stratify=train_df['label']
)

**4: Tokenizer & Dataset**

In [9]:
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

class HateDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = tokenizer(
            self.texts[idx],
            truncation=True,
            padding="max_length",
            max_length=128,
            return_tensors="pt"
        )
        item = {k: v.squeeze() for k, v in encoding.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

train_dataset = HateDataset(X_train.tolist(), y_train.tolist())
val_dataset = HateDataset(X_val.tolist(), y_val.tolist())


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

**5: Load mBERT Model**

In [10]:
model = BertForSequenceClassification.from_pretrained(
    "bert-base-multilingual-cased",
    num_labels=2
)


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


**6: Training Arguments**

In [13]:
training_args = TrainingArguments(
    output_dir="./mbert_results",
    do_eval=True,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_steps=50,
    load_best_model_at_end=True
)

**7: Train**

In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)

trainer.train()


/tmp/ipython-input-3837116936.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 3


wandb: You chose "Don't visualize my results"


Epoch,Training Loss,Validation Loss
1,0.312800,0.312676
2,0.353600,0.318896


TrainOutput(global_step=316, training_loss=0.3207178040395809, metrics={'train_runtime': 300.2679, 'train_samples_per_second': 16.798, 'train_steps_per_second': 1.052, 'total_flos': 331783040808960.0, 'train_loss': 0.3207178040395809, 'epoch': 2.0})

**8: Evaluation**

In [15]:
preds = trainer.predict(val_dataset)
y_pred = np.argmax(preds.predictions, axis=1)

print(classification_report(y_val, y_pred))
print("Macro F1:", f1_score(y_val, y_pred, average="macro"))


              precision    recall  f1-score   support

           0       0.90      1.00      0.95       570
           1       0.00      0.00      0.00        61

    accuracy                           0.90       631
   macro avg       0.45      0.50      0.47       631
weighted avg       0.82      0.90      0.86       631

Macro F1: 0.4746044962531224


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
